<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
! pip -q install transformers
! pip -q install parsivar

     |████████████████████████████████| 1.1MB 8.5MB/s 
     |████████████████████████████████| 890kB 52.5MB/s 
     |████████████████████████████████| 1.1MB 52.7MB/s 
     |████████████████████████████████| 3.0MB 54.7MB/s 
     |████████████████████████████████| 36.2MB 83kB/s 
     |████████████████████████████████| 1.5MB 22.3MB/s 


In [30]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
import re
parsivar_tokenizer = Tokenizer()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()
print()

# **Tokenize**

### Example

In [4]:
tweet_example = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

In [5]:
input_sentence = tokenizer.tokenize(tweet_example)
print(input_sentence)

['بابا', 'بیخیال', ':', ')', ')', 'همون', 'سال', 'اتلتیکو', 'با', 'گل', 'افساید', 'به', 'بایرن', 'رفت', 'فینال', '.', 'مشکل', 'داوری', 'نداشتیم', 'اون', 'بازی', 'که', 'میگی', '.', 'فصل', 'قبلش', 'هم', '۴', '۰', 'با', 'هاینکس', 'زدیم', 'شما', 'رو']


### Tokenize All tweets

In [6]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

--2020-10-02 20:31:23--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2051971 (2.0M) [text/plain]
Saving to: ‘extracted_data.txt’

extracted_data.txt  100%[===================>]   1.96M  --.-KB/s    in 0.06s   

2020-10-02 20:31:24 (30.3 MB/s) - ‘extracted_data.txt’ saved [2051971/2051971]



In [7]:
! head -n 40 extracted_data.txt

  سلام
خواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. 
چند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.

**************

ولادتِ حضرت امام موسی کاظمؑ 

**************

iran iranelection  - و صدای شیون فرزندانت که وحشت زده گریه می‌کردند؛ نوشته‌ی مهرزاد محب‌پور همسر مژگان عبادی بهای... 

**************

 Iran IranElection صبا محب‌پور، جوان محروم از تحصیل بهایی: «قانون، تنها عنوانی حک شده در کنار نام سرزمین من ایران است»

**************

  محسن رضایی هم یک جوان ۲۶،۲۷ ساله بوده
مدیریت جنگ با مدیریت فعلی کشور که اکثرا افراد لیبرابی هستن که در جنگ هیچ نقشی نداشتن و یا نهایتا پشت جبهه مشغول بودن قابل مقایسه نیست

**************

 حسین باستانی یه مستند ساخته به اسم کودتای خزنده که بر اساس نوار درز کرده یه جلسه یه سری فرمانده‌ها در وسط جنگه با محسن رضایی (فرمانده کل سپاه تو اون زمان). تو اون جلسه فرمانده‌ها کلی شاکی‌ن از بی‌کفایتی یه سری از رده‌بالایی‌ها و باق

In [8]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'  سلام\nخواهر من فکر '

In [9]:
tweets = tweets_str.split("\n\n**************\n\n")
tweets[0], len(tweets)

('  سلام\nخواهر من فکر کنم دو سالی سرفه امانش را بریده بود.خیلی کارها کرد برای درمانش اما درست نشد که نشد. کرونا هم که شروع شد، خیلی نگرانش بودیم. \nچند روز پیش با چند تا نیش زنبور عسل به اصطلاح زنبور درمانی کرد و با کمال تعجب دیگه سرفه نمی کنه.',
 6978)

In [10]:
len(tweets)

6978

In [ ]:
len(set(tweets))

6543

In [ ]:
# tweets_tokens = []
# for tweet in tweets:
#   tweet_tokens = tokenizer.tokenize(tweet)
#   tweets_tokens.append(tweet_tokens)

In [41]:
tweets_tokens = []
for tweet in set(tweets):
  tweet_tokens = re.sub(' +', ' ', tweet.replace('\n', ' ').replace('.', ' ')).split(' ')
  tweets_tokens.append(tweet_tokens)

In [50]:
tweets_tokens[1]

['',
 'متاسفم',
 '،',
 'به',
 'امید',
 'اصلاح',
 'حداقلی',
 'سعی',
 'در',
 'مشارکت',
 'داشتم',
 'چون',
 'هزینه',
 'اصلاح',
 'را',
 'برای',
 'کشور',
 'به',
 'صرفه',
 'تر',
 'میدیدم',
 'ولی',
 'صد',
 'افسوس',
 'که',
 'علاقه',
 'این',
 'جماعت',
 'به',
 'ثروت',
 'و',
 'قدرت',
 'بی',
 'حد',
 'و',
 'حصر',
 'امیدمان',
 'را',
 'به',
 'اصلاح',
 'ناامید',
 'کرد،متاسفم',
 'که',
 'این',
 'سیستم',
 'اصلاح',
 'پذیر',
 'نیست',
 'همه',
 'بازی',
 'گرانش',
 '،نقش',
 'بازی',
 'می\u200cکنند']

In [51]:
counter = 0
with open('persian-ner-twitter-data.txt', 'w') as file:
  for tweet_tokens in tweets_tokens:
    for token in tweet_tokens:
      counter += 1
      file.write(token + '\tO\n')
    file.write('\n')
print(counter)

192468


In [52]:
! head -n 30 persian-ner-twitter-data.txt

	O

	O
متاسفم	O
،	O
به	O
امید	O
اصلاح	O
حداقلی	O
سعی	O
در	O
مشارکت	O
داشتم	O
چون	O
هزینه	O
اصلاح	O
را	O
برای	O
کشور	O
به	O
صرفه	O
تر	O
میدیدم	O
ولی	O
صد	O
افسوس	O
که	O
علاقه	O
این	O
جماعت	O


# **Model**

In [ ]:
from transformers import pipeline

In [ ]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet_example)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-organization',
  'index': 8,
  'score': 0.9995940327644348,
  'word': 'اتلتیکو'},
 {'entity': 'B-organization',
  'index': 13,
  'score': 0.9972712397575378,
  'word': 'بایرن'},
 {'entity': 'B-person',
  'index': 31,
  'score': 0.9915841817855835,
  'word': 'هاینکس'}]